In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pelita.ipython

In [2]:
application, sockets = pelita.ipython.printit()

Listening on 0.0.0.0, port 51785


In [3]:
application.path

'ipc:///tmp/pelita.139668139983872'

In [7]:
from IPython.display import IFrame
url = "http://%s:%i/static/index.html" % sockets[0].getsockname()
print(url)
IFrame(url, width=600, height=300)

WebSocket closed
http://0.0.0.0:51785/static/index.html


In [5]:
from subprocess import Popen

WebSocket opened


In [8]:
Popen(["/home/schuppner/Projects/pelita/pelitagame", "--null", "--publish", application.path])

WebSocket opened


ERROR:tornado.application:Uncaught exception GET /ws-echo (127.0.0.1)
HTTPServerRequest(protocol='http', host='0.0.0.0:51785', method='GET', uri='/ws-echo', version='HTTP/1.1', remote_ip='127.0.0.1', headers={'Origin': 'http://0.0.0.0:51785', 'Connection': 'Upgrade', 'Sec-Websocket-Version': '13', 'Cache-Control': 'no-cache', 'Accept-Encoding': 'gzip, deflate, sdch', 'Pragma': 'no-cache', 'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36', 'Dnt': '1', 'Sec-Websocket-Extensions': 'permessage-deflate; client_max_window_bits', 'Sec-Websocket-Key': 'HzLsEPYtCCPiWji+qXPe5Q==', 'Host': '0.0.0.0:51785', 'Upgrade': 'websocket', 'Accept-Language': 'en-US,en;q=0.8,de;q=0.6'})
Traceback (most recent call last):
  File "/home/schuppner/.local/lib/python3.4/site-packages/tornado/web.py", line 1302, in _stack_context_handle_exception
    raise_exc_info((type, value, traceback))
  File "<string>", line 3, in raise_exc_info
  File 

In [11]:
application2, sockets2 = pelita.ipython.printit()

Listening on 0.0.0.0, port 36979


In [12]:
IFrame("http://%s:%i/static/index.html" % sockets2[0].getsockname(), width=800, height=500)

In [16]:
Popen(["/home/schuppner/Projects/pelita/pelitagame", "--null", "--publish", application2.path])